In [1]:
import pandas as pd
import numpy as np

#read data
data = pd.read_csv('data/train.csv',
                    usecols = ['Name','OutcomeType','AnimalType','SexuponOutcome',
                    'AgeuponOutcome','Breed','Color'])
data = pd.DataFrame(data)

# print(data.isnull().sum())
data['Name'].fillna(value = 0, inplace=True) # Name缺7691個，補0
data['AgeuponOutcome'].fillna(value = 7300, inplace=True) # age缺18個，補mean
data['SexuponOutcome'].fillna('Neutered Male', inplace=True) # Since there is only one NA, I will assign it to maximum class

order = {'Adoption':0,'Died':1,'Euthanasia':2,'Return_to_owner':3,'Transfer':4}

OutcomeType = data['OutcomeType'].tolist()
for index,type_ in enumerate(OutcomeType):
    OutcomeType[index] = order[type_]
OutcomeType = pd.DataFrame(OutcomeType)
data['OutcomeType'] = OutcomeType
data

,Name,OutcomeType,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,Hambone,3,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,Emily,2,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,Pearce,0,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,0,4,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,0,4,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan
...,...,...,...,...,...,...,...
26724,0,4,Cat,Intact Male,1 month,Domestic Shorthair Mix,Brown Tabby/White
26725,0,4,Cat,Spayed Female,3 months,Domestic Shorthair Mix,Brown Tabby
26726,Zeus,0,Dog,Neutered Male,4 years,Old English Bulldog Mix,White/Tan
26727,0,4,Cat,Intact Male,4 weeks,Domestic Shorthair Mix,Black


In [2]:
# Age
age = np.array(data['AgeuponOutcome'])

def age_to_days(item):
    # convert item to list if it is one string
    if type(item) is str:
        item = [item]
    ages_in_days = np.zeros(len(item))
    for i in range(len(item)):
        # check if item[i] is str
        if type(item[i]) is str:
            if 'day' in item[i]:
                ages_in_days[i] = int(item[i].split(' ')[0])
            if 'week' in item[i]:
                ages_in_days[i] = int(item[i].split(' ')[0])*7
            if 'month' in item[i]:
                ages_in_days[i] = int(item[i].split(' ')[0])*30
            if 'year' in item[i]:
                ages_in_days[i] = int(item[i].split(' ')[0])*365    
        else:
            # item[i] is not a string but a nan
            ages_in_days[i] = 794
    return ages_in_days

day= age_to_days(age)

# print(age_to_days(np.array(data['AgeuponOutcome'])).mean())    平均是794.1307700947175，拿去補前面18個缺值
day= {'day':day}
day= pd.DataFrame(day)
data['AgeuponOutcome'] = day

day

,day
0,365.0
1,365.0
2,730.0
3,21.0
4,730.0
...,...
26724,30.0
26725,90.0
26726,1460.0
26727,28.0


In [3]:
# gender - One hot encoding
gender = data['SexuponOutcome']
gender_dict={}
uni_gender = np.unique(gender).tolist()
for index,perGender in enumerate(uni_gender):
    gender_dict.setdefault(perGender,index)

gender_calss = []
for index,sex in enumerate(gender.tolist()):
    gender_calss.append(gender_dict[sex])

gender = pd.get_dummies(gender)
gender = pd.DataFrame(gender)
data = pd.concat([data,gender],axis=1)

gender

,Intact Female,Intact Male,Neutered Male,Spayed Female,Unknown
0,0,0,1,0,0
1,0,0,0,1,0
2,0,0,1,0,0
3,0,1,0,0,0
4,0,0,1,0,0
...,...,...,...,...,...
26724,0,1,0,0,0
26725,0,0,0,1,0
26726,0,0,1,0,0
26727,0,1,0,0,0


In [4]:
# breed
import re
data['Breed'] = data.Breed.str.extract('({})'.format('|'.join(['Mix'])), 
                        flags=re.IGNORECASE, expand = False).str.lower().fillna('pure')

data['Breed'] = data['Breed'].apply(lambda x : 1 if 'mix' in x else 0)

In [5]:
#貓狗
animalType  = data['AnimalType']
animalType = pd.get_dummies(animalType)
data = pd.concat([data,animalType],axis=1)
data = data.drop(['AnimalType','SexuponOutcome'],axis=1)
data 

,Name,OutcomeType,AgeuponOutcome,Breed,Color,Intact Female,Intact Male,Neutered Male,Spayed Female,Unknown,Cat,Dog
0,Hambone,3,365.0,1,Brown/White,0,0,1,0,0,0,1
1,Emily,2,365.0,1,Cream Tabby,0,0,0,1,0,1,0
2,Pearce,0,730.0,1,Blue/White,0,0,1,0,0,0,1
3,0,4,21.0,1,Blue Cream,0,1,0,0,0,1,0
4,0,4,730.0,0,Tan,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
26724,0,4,30.0,1,Brown Tabby/White,0,1,0,0,0,1,0
26725,0,4,90.0,1,Brown Tabby,0,0,0,1,0,1,0
26726,Zeus,0,1460.0,1,White/Tan,0,0,1,0,0,0,1
26727,0,4,28.0,1,Black,0,1,0,0,0,1,0


In [6]:
# 是否為multi Color
color = data['Color'].apply(lambda x : 1 if '/' in x else 0)
data['Color'] = color

color

0        1
1        0
2        1
3        0
4        0
        ..
26724    1
26725    0
26726    1
26727    0
26728    1
Name: Color, Length: 26729, dtype: int64

In [7]:
# name分有沒有名字
name = data['Name']
name.values[name != 0] = 1
name = name.astype('int8')
data['Name'] = name

print(name)       #dog有名字的話是1，沒名字是0
data

0        1
1        1
2        1
3        0
4        0
        ..
26724    0
26725    0
26726    1
26727    0
26728    0
Name: Name, Length: 26729, dtype: int8


,Name,OutcomeType,AgeuponOutcome,Breed,Color,Intact Female,Intact Male,Neutered Male,Spayed Female,Unknown,Cat,Dog
0,1,3,365.0,1,1,0,0,1,0,0,0,1
1,1,2,365.0,1,0,0,0,0,1,0,1,0
2,1,0,730.0,1,1,0,0,1,0,0,0,1
3,0,4,21.0,1,0,0,1,0,0,0,1,0
4,0,4,730.0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
26724,0,4,30.0,1,1,0,1,0,0,0,1,0
26725,0,4,90.0,1,0,0,0,0,1,0,1,0
26726,1,0,1460.0,1,1,0,0,1,0,0,0,1
26727,0,4,28.0,1,0,0,1,0,0,0,1,0


In [8]:
# OutcomeType - LabelEncode
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(data['OutcomeType'])
label_df = pd.DataFrame({'label': encoded_labels})

label_df

,label
0,3
1,2
2,0
3,4
4,4
...,...
26724,4
26725,4
26726,0
26727,4


In [9]:
#read data
test_data = pd.read_csv('data/test.csv',
                    usecols = ['ID','Name','AnimalType','SexuponOutcome',
                    'AgeuponOutcome','Breed','Color'])



test_data['Breed'] = test_data.Breed.str.extract('({})'.format('|'.join(['Mix'])), 
                        flags=re.IGNORECASE, expand = False).str.lower().fillna('pure')
test_data['Breed'] = test_data['Breed'].apply(lambda x : 1 if 'mix' in x else 0)

test_data['Color'].apply(lambda x : 1 if '/' in x else 0)
gender = test_data['SexuponOutcome']
gender = pd.get_dummies(gender)
gender = pd.DataFrame(gender)
test_data = pd.concat([test_data,gender],axis=1)
name = test_data['Name']
name.values[name != 0] = 1
name = name.astype('int8')
test_data['Name'] = name
animalType  = test_data['AnimalType']
animalType = pd.get_dummies(animalType)
test_data = pd.concat([test_data,animalType],axis=1)
test_data = test_data.drop(['AnimalType','SexuponOutcome'],axis=1)

# multi Color
color = test_data['Color'].apply(lambda x : 1 if '/' in x else 0)
test_data['Color'] = color

age = np.array(test_data['AgeuponOutcome'])
def age_to_days(item):
    # convert item to list if it is one string
    if type(item) is str:
        item = [item]
    ages_in_days = np.zeros(len(item))
    for i in range(len(item)):
        # check if item[i] is str
        if type(item[i]) is str:
            if 'day' in item[i]:
                ages_in_days[i] = int(item[i].split(' ')[0])
            if 'week' in item[i]:
                ages_in_days[i] = int(item[i].split(' ')[0])*7
            if 'month' in item[i]:
                ages_in_days[i] = int(item[i].split(' ')[0])*30
            if 'year' in item[i]:
                ages_in_days[i] = int(item[i].split(' ')[0])*365    
    return ages_in_days

test_day= age_to_days(age)
test_day= {'day':test_day}
test_day= pd.DataFrame(test_day)
test_data['AgeuponOutcome'] = test_day

test_data = pd.DataFrame(test_data)

In [10]:
import os
folder_name = 'preprocess_done'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
data.to_csv((os.path.join(folder_name, 'train_preprocess_done.csv')), index=False)
test_data.to_csv((os.path.join(folder_name, 'test_preprocess_done.csv')), index=False)